In [2]:
!pip install nltk

In [4]:
import pandas as pd
import numpy as np
import random
import joblib
import torch

from sklearn import preprocessing
from sklearn import model_selection

# from transformers import AdamW
# from transformers import get_linear_schedule_with_warmup
from nltk.corpus import wordnet
import config
# from transformers import XLMRobertaForTokenClassification, XLMRobertaConfig ,BertModel, XLMRobertaTokenizer, XLMRobertaModel, BertForTokenClassification
from data_utils import loadDatafromFile,createTokenizedDf,CompDataset,createkfoldData,createDataloaders
from seqeval.metrics import accuracy_score, classification_report
import engine
import torch.nn as nn
from tqdm import tqdm

In [40]:
class DataAugmentation:
    def __init__(self):
        self.stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 
    'ours', 'ourselves', 'you', 'your', 'yours', 
    'yourself', 'yourselves', 'he', 'him', 'his', 
    'himself', 'she', 'her', 'hers', 'herself', 
    'it', 'its', 'itself', 'they', 'them', 'their', 
    'theirs', 'themselves', 'what', 'which', 'who', 
    'whom', 'this', 'that', 'these', 'those', 'am', 
    'is', 'are', 'was', 'were', 'be', 'been', 'being', 
    'have', 'has', 'had', 'having', 'do', 'does', 'did',
    'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or',
    'because', 'as', 'until', 'while', 'of', 'at', 
    'by', 'for', 'with', 'about', 'against', 'between',
    'into', 'through', 'during', 'before', 'after', 
    'above', 'below', 'to', 'from', 'up', 'down', 'in',
    'out', 'on', 'off', 'over', 'under', 'again', 
    'further', 'then', 'once', 'here', 'there', 'when', 
    'where', 'why', 'how', 'all', 'any', 'both', 'each', 
    'few', 'more', 'most', 'other', 'some', 'such', 'no', 
    'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 
    'very', 's', 't', 'can', 'will', 'just', 'don', 
    'should', 'now', '']
    #DataAugmentation methods
    
    def swap_word(self,new_words,labels_src):
        '''Helper function for random swap.''' 

        random_idx_1 = random.randint(0, len(new_words)-1)
        random_idx_2 = random_idx_1
        counter = 0
        while random_idx_2 == random_idx_1:
            random_idx_2 = random.randint(0, len(new_words)-1)
            counter += 1
            if counter > 3:
                return (new_words,labels_src)
        new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
        labels_src[random_idx_1], labels_src[random_idx_2] = labels_src[random_idx_2], labels_src[random_idx_1]
        return (new_words, labels_src)

    
    def random_swap(self,dataframe,n): # n is number of times to swap randomly 2 words

        '''Takes in input the initial dataframe read from the files and returns 
        modefied/increased dataframe with swapped word fro each source sentence and its corresponding token wise labels '''
        
        source_sentences  = list(dataframe.source)
        target_sentences = list(dataframe.target)
        labels_src = list(dataframe.src_tokens)
        labels_tar = list(dataframe.tar_tokens)
        source_sentences_temp =[]
        labels_sec_temp=[]
        i=0

        for sentences, labels in zip(source_sentences,labels_src):

            sentences = sentences.split()
            labels = labels.split()
            for _ in range(n):
                sentences, labels = self.swap_word(sentences,labels)

            sentences_str = ' '.join(sentences)
            labels_str = ' '.join(labels)
            target_sentences.append(target_sentences[i])
            labels_tar.append(labels_tar[i])
            source_sentences_temp.append(sentences_str)
            labels_sec_temp.append(labels_str)
    #         break
            i+=1

        source_sentences.extend(source_sentences_temp)
        labels_src.extend(labels_sec_temp)

        column_names = ["source","target","src_tokens","tar_tokens"]
        df = pd.DataFrame(columns=column_names,dtype=object)
        df = df.assign(source=source_sentences)
        df = df.assign(target = target_sentences)
        df = df.assign(src_tokens = labels_src)
        df = df.assign(tar_tokens = labels_tar)

        return df


    
    def random_deletion(self,dataframe, p):
        
        '''Takes input dataframe created after reading data files and the probabaility for deletion of random tokens in the
        source sentences and returns increased dataframe with combined orignal sentences and noisy sentences '''
    
        source_sentences  = list(dataframe.source)
        target_sentences = list(dataframe.target)
        labels_src = list(dataframe.src_tokens)
        labels_tar = list(dataframe.tar_tokens)
        senetences_temp=[]
        labels_temp= []
        #randomly delete words with probability p
        i=0
        for sentences, labels in zip(source_sentences,labels_src):
            sentences = sentences.split()
            labels = labels.split() 
            source_sentences_temp=[]
            labels_sec_temp=[]
            if len(sentences) == 1:
                pass
            for word,label in zip(sentences,labels):
                r = random.uniform(0, 1)
                if r > p:
                    source_sentences_temp.append(word)
                    labels_sec_temp.append(label)
            if len(source_sentences_temp) == 0: #if you end up deleting all words, just return a random word
                rand_int = random.randint(0, len(source_sentences_temp)-1)
                source_sentences_temp.append(sentences[rand_int])
                labels_sec_temp.append(labels[rand_int])

            sentences_str = ' '.join(source_sentences_temp)
            labels_str = ' '.join(labels_sec_temp)
            senetences_temp.append(sentences_str)
            labels_temp.append(labels_str)
            target_sentences.append(target_sentences[i])
            labels_tar.append(labels_tar[i])
    #         break
            i+=1
        source_sentences.extend(senetences_temp)
        labels_src.extend(labels_temp)    
        column_names = ["source","target","src_tokens","tar_tokens"]
        df = pd.DataFrame(columns=column_names,dtype=object)
        df = df.assign(source=source_sentences)
        df = df.assign(target = target_sentences)
        df = df.assign(src_tokens = labels_src)
        df = df.assign(tar_tokens = labels_tar)  


        return df
    

    def get_synonyms(self,word):
        
        '''Helper function for synonym replacement '''
        
        synonyms = set()
        for syn in wordnet.synsets(word): 
            for l in syn.lemmas(): 
                synonym = l.name().replace("_", " ").replace("-", " ").lower()
                synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
                synonyms.add(synonym) 
        if word in synonyms:
            synonyms.remove(word)
        return list(synonyms)
    
    
    def synonym_replacement(self,dataframe, n):
    #     new_words = words.copy()
        source_sentences  = list(dataframe.source)
        target_sentences = list(dataframe.target)
        labels_src = list(dataframe.src_tokens)
        labels_tar = list(dataframe.tar_tokens)
        senetences_temp=[]
        labels_temp= []
        k=0
        for sentences,labels in zip(source_sentences,labels_src):
            dict={}
            sentences = sentences.split() # ['tarun', 'are', 'bad', 'for','health']
            labels = labels.split()#['OK','BAD','OK','OK','OK']
            for i, words in enumerate(sentences) :
                dict[words] = i                      # {tarun:0, is : 1 , bad: 3 , for : 4, health : 5}
            random_word_list = list(set([word for word in sentences if word not in self.stop_words]))
            random.shuffle(random_word_list) # ['bad', 'are'...]
            num_replaced = 0
            for random_word in random_word_list: # ['bad', 'are'...]
                synonyms = get_synonyms(random_word)
                if len(synonyms) >= 1:
                    synonym = random.choice(list(synonyms)) # bad -- > not good
                    sentences = [synonym if word == random_word else word for word in sentences] # ['tarun', 'are', 'not good', 'for','health']
                    synonym_len = len(synonym.split()) # 2 
                    index_random_word = dict[random_word] # 2
                    flag=0
                    for key,values in dict.items():
                        if key == random_word: # {tarun:0, is : 1 , bad: 2 , for : 3, health : 4}
                            flag+=1
                        if flag ==1:
                            dict[key] = values+synonym_len-1 # {tarun:0, is : 1 , bad: 3 , for : 4, health : 5}
                    label_synonym = labels[index_random_word] # 2 --> OK
                    labels_append_synonym = [label_synonym] * synonym_len # ['OK','OK']
                    labels_1 = labels[:index_random_word] # ['OK','BAD']
                    labels_2 = labels[index_random_word+1:]#['OK','OK']
                    labels_1.extend(labels_append_synonym)# ['OK','BAD','OK','OK']
                    labels_1.extend(labels_2)# ['OK','BAD','OK','OK','OK','OK']
                    labels = labels_1
                    num_replaced += 1

                if num_replaced >= n: #only replace up to n words
                    break
            sentence = ' '.join(sentences)
            labels_splt = ' '.join(labels)
            senetences_temp.append(sentence)
            labels_temp.append(labels_splt)
            target_sentences.append(target_sentences[k])
            labels_tar.append(labels_tar[k])
    #         break
            k+=1 

        source_sentences.extend(senetences_temp)
        labels_src.extend(labels_temp)
        column_names = ["source","target","src_tokens","tar_tokens"]
        df = pd.DataFrame(columns=column_names,dtype=object)
        df = df.assign(source=source_sentences)
        df = df.assign(target = target_sentences)
        df = df.assign(src_tokens = labels_src)
        df = df.assign(tar_tokens = labels_tar)

        return df


In [5]:

stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 
    'ours', 'ourselves', 'you', 'your', 'yours', 
    'yourself', 'yourselves', 'he', 'him', 'his', 
    'himself', 'she', 'her', 'hers', 'herself', 
    'it', 'its', 'itself', 'they', 'them', 'their', 
    'theirs', 'themselves', 'what', 'which', 'who', 
    'whom', 'this', 'that', 'these', 'those', 'am', 
    'is', 'are', 'was', 'were', 'be', 'been', 'being', 
    'have', 'has', 'had', 'having', 'do', 'does', 'did',
    'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or',
    'because', 'as', 'until', 'while', 'of', 'at', 
    'by', 'for', 'with', 'about', 'against', 'between',
    'into', 'through', 'during', 'before', 'after', 
    'above', 'below', 'to', 'from', 'up', 'down', 'in',
    'out', 'on', 'off', 'over', 'under', 'again', 
    'further', 'then', 'once', 'here', 'there', 'when', 
    'where', 'why', 'how', 'all', 'any', 'both', 'each', 
    'few', 'more', 'most', 'other', 'some', 'such', 'no', 
    'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 
    'very', 's', 't', 'can', 'will', 'just', 'don', 
    'should', 'now', '']
def synonym_replacement(dataframe, n):
#     new_words = words.copy()
    source_sentences  = list(dataframe.source)
    target_sentences = list(dataframe.target)
    labels_src = list(dataframe.src_tokens)
    labels_tar = list(dataframe.tar_tokens)
    senetences_temp=[]
    labels_temp= []
    k=0
    for sentences,labels in zip(source_sentences,labels_src):
        dict={}
        sentences = sentences.split()
        labels_src = labels.split()
#         for i, words in enumerate(sentences) :
#             dict[words] = i
#         print(dict)
        random_word_list = list(set([word for word in sentences if word not in stop_words]))
        random.shuffle(random_word_list)
        print(random_word_list)
        num_replaced = 0
        for random_word in random_word_list:
#             print(random_word)
            synonyms = get_synonyms(random_word)
#             print(synonyms)
            if len(synonyms) >= 1:
                synonym = random.choice(list(synonyms))
#                 print(synonym)
                sentences = [synonym if word == random_word else word for word in sentences]
                #print("replaced", random_word, "with", synonym)
#                 print(labels_src)
                synonym_len = len(synonym.split())
                
                index_replaced_word = [i for i,item in enumerate(sentences)]
                
#                 index_random_word = dict[random_word]
#                 print(index_random_word)
                labels_replaced_word = [labes_src[i] for i in index_replaced_word]
            
                label_synonym = labels_src[index_random_word]
#                 print(label_synonym)
                labels_append_synonym = [label_synonym] * synonym_len
#                 print(labels_append_synonym)
                labels_1 = labels_src[:index_random_word]
#                 print(labels_1)
                labels_2 = labels_src[index_random_word+1:]
#                 print(labels_2)
                labels_1.extend(labels_append_synonym)
#                 print(labels_1)
                labels_1.extend(labels_2)
#                 print(labels_1)
                
                num_replaced += 1
  
            if num_replaced >= n: #only replace up to n words
                break
        sentence = ' '.join(sentences)
        labels_splt = ' '.join(labels_1)
        senetences_temp.append(sentence)
        labels_temp.append(labels_splt)
        target_sentences.append(target_sentences[k])
        labels_tar.append(labels_tar[k])
        k+=1     
#         sentence = ' '.join(sentences)
#     print(sentence)
#         new_words = sentence.split(' ')

    return new_words

def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word): 
        for l in syn.lemmas(): 
            synonym = l.name().replace("_", " ").replace("-", " ").lower()
            synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
            synonyms.add(synonym) 
    if word in synonyms:
        synonyms.remove(word)
    return list(synonyms)

In [6]:
dataObj = loadDatafromFile(config.filePath_src,config.filePath_tar, config.filePath_srcTags,config.filePath_tarTags)
df= dataObj.createDf() 
df

,source,target,src_tokens,tar_tokens
0,José Ortega y Gasset visited Husserl at Freibu...,1934 besuchte José Ortega y Gasset Husserl in ...,OK OK OK OK OK OK OK OK OK OK OK,OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK O...
1,"However , a disappointing ninth in China meant...",Eine enttäuschende Neunte in China bedeutete j...,OK OK OK BAD OK OK OK OK OK OK OK OK OK OK OK ...,OK BAD OK BAD OK BAD OK OK OK OK OK OK OK OK O...
2,"In his diary , Chase wrote that the release of...","In seinem Tagebuch , Chase schrieb , dass die ...",OK OK OK OK BAD BAD OK OK OK OK OK OK OK OK BA...,OK OK OK OK OK OK OK OK OK BAD OK BAD OK OK OK...
3,Heavy arquebuses mounted on wagons were called...,Schwere Arquebuses auf Waggons montiert wurden...,OK BAD BAD OK OK OK OK OK OK OK OK,OK OK OK BAD OK OK OK OK BAD BAD OK OK OK OK O...
4,Once North Pacific salmon die off after spawni...,Sobald der nordpazifische Lachs nach dem Laich...,OK OK OK OK BAD OK OK OK OK OK BAD OK BAD OK O...,OK OK OK BAD OK OK OK BAD OK OK OK OK OK OK OK...
...,...,...,...,...
6995,Some may also discourage or disallow unsanitar...,Einige können auch unhygienische Praktiken wie...,OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK O...,OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK O...
6996,"In the late 1860s , the crinolines disappeared...",In den späten 1860er Jahren verschwanden die K...,OK OK OK OK OK OK OK OK OK OK BAD BAD OK OK OK...,OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK O...
6997,"Disco was criticized as mindless , consumerist...","Disco wurde als geistlos , konsumistisch , übe...",OK OK OK OK BAD OK BAD OK OK OK OK OK,OK OK OK OK OK OK OK BAD OK OK OK BAD OK OK OK...
6998,Planters would then fill large hogsheads with ...,Die Pflanzer würden dann große Heuschrecken mi...,OK OK OK BAD OK BAD OK OK BAD BAD OK OK OK OK OK,OK OK OK OK OK BAD OK OK OK OK OK BAD OK OK OK...


In [33]:


# def synonym_replacement(dataframe, n):
# #     new_words = words.copy()
#     source_sentences  = list(dataframe.source)
#     target_sentences = list(dataframe.target)
#     labels_src = list(dataframe.src_tokens)
#     labels_tar = list(dataframe.tar_tokens)
#     senetences_temp=[]
#     labels_temp= []
#     k=0
#     for sentences,labels in zip(source_sentences,labels_src):
#         dict={}
#         sentences = sentences.split() # ['tarun', 'are', 'bad', 'for','health']
#         labels = labels.split()#['OK','BAD','OK','OK','OK']
#         for i, words in enumerate(sentences) :
#             dict[words] = i                      # {tarun:0, is : 1 , bad: 3 , for : 4, health : 5}
#         random_word_list = list(set([word for word in sentences if word not in config.stop_words]))
#         random.shuffle(random_word_list) # ['bad', 'are'...]
#         num_replaced = 0
#         for random_word in random_word_list: # ['bad', 'are'...]
#             synonyms = get_synonyms(random_word)
#             if len(synonyms) >= 1:
#                 synonym = random.choice(list(synonyms)) # bad -- > not good
#                 sentences = [synonym if word == random_word else word for word in sentences] # ['tarun', 'are', 'not good', 'for','health']
#                 synonym_len = len(synonym.split()) # 2 
#                 index_random_word = dict[random_word] # 2
#                 flag=0
#                 for key,values in dict.items():
#                     if key == random_word: # {tarun:0, is : 1 , bad: 2 , for : 3, health : 4}
#                         flag+=1
#                     if flag ==1:
#                         dict[key] = values+synonym_len-1 # {tarun:0, is : 1 , bad: 3 , for : 4, health : 5}
#                 label_synonym = labels[index_random_word] # 2 --> OK
#                 labels_append_synonym = [label_synonym] * synonym_len # ['OK','OK']
#                 labels_1 = labels[:index_random_word] # ['OK','BAD']
#                 labels_2 = labels[index_random_word+1:]#['OK','OK']
#                 labels_1.extend(labels_append_synonym)# ['OK','BAD','OK','OK']
#                 labels_1.extend(labels_2)# ['OK','BAD','OK','OK','OK','OK']
#                 labels = labels_1
#                 num_replaced += 1
  
#             if num_replaced >= n: #only replace up to n words
#                 break
#         sentence = ' '.join(sentences)
#         labels_splt = ' '.join(labels)
#         senetences_temp.append(sentence)
#         labels_temp.append(labels_splt)
#         target_sentences.append(target_sentences[k])
#         labels_tar.append(labels_tar[k])
# #         break
#         k+=1 
         
#     source_sentences.extend(senetences_temp)
#     labels_src.extend(labels_temp)
#     column_names = ["source","target","src_tokens","tar_tokens"]
#     df = pd.DataFrame(columns=column_names,dtype=object)
#     df = df.assign(source=source_sentences)
#     df = df.assign(target = target_sentences)
#     df = df.assign(src_tokens = labels_src)
#     df = df.assign(tar_tokens = labels_tar)

#     return df


# def get_synonyms(word):
#     synonyms = set()
#     for syn in wordnet.synsets(word): 
#         for l in syn.lemmas(): 
#             synonym = l.name().replace("_", " ").replace("-", " ").lower()
#             synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
#             synonyms.add(synonym) 
#     if word in synonyms:
#         synonyms.remove(word)
#     return list(synonyms)

In [41]:
dataObj = loadDatafromFile(config.filePath_src,config.filePath_tar, config.filePath_srcTags,config.filePath_tarTags)
df= dataObj.createDf() 
augObj = DataAugmentation()
augObj.synonym_replacement(df,2)

,source,target,src_tokens,tar_tokens
0,José Ortega y Gasset visited Husserl at Freibu...,1934 besuchte José Ortega y Gasset Husserl in ...,OK OK OK OK BAD OK OK OK BAD BAD OK,OK BAD OK BAD OK OK OK OK OK OK OK OK BAD OK O...
1,"however , a disappointing ninth in China meant...",eine enttäuschende Neunte in China bedeutete j...,OK BAD BAD BAD BAD OK OK OK OK OK OK OK OK OK ...,BAD BAD OK BAD OK BAD OK OK OK OK OK OK OK OK ...
2,"in his diary , Chase wrote that the release of...","in seinem Tagebuch , Chase schrieb , dass die ...",OK OK OK OK OK BAD OK OK BAD OK OK BAD OK BAD ...,OK OK OK OK OK OK OK BAD OK OK OK BAD OK OK OK...
3,Heavy arquebuses mounted on wagons were called...,schwere Arquebuses auf Waggons montiert wurden...,BAD BAD OK OK BAD BAD BAD OK OK OK OK,OK OK BAD BAD OK OK OK BAD OK OK BAD OK BAD OK...
4,once North Pacific salmon die off after spawni...,sobald der nordpazifische Lachs nach dem Laich...,OK OK OK OK OK OK OK BAD OK BAD BAD BAD BAD OK...,OK OK OK OK OK OK OK OK OK OK OK OK OK OK BAD ...
...,...,...,...,...
13995,some may too discourage or disallow insanitary...,einige können auch unhygienische Praktiken wie...,OK OK OK OK OK OK OK OK OK OK OK OK OK OK BAD ...,OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK O...
13996,"in the late 1860s , the crinolines disappeared...",in den späten 1860er Jahren verschwanden die K...,OK OK OK OK OK OK OK BAD OK BAD BAD BAD OK OK ...,OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK B...
13997,"disco was knock as mindless , consumerist , ov...","Disco wurde als geistlos , konsumistisch , übe...",OK OK OK OK BAD BAD OK OK OK BAD OK OK,OK OK OK OK OK OK OK BAD OK OK OK BAD OK OK OK...
13998,planters would then satisfy large hogsheads wi...,die Pflanzer würden dann große Heuschrecken mi...,OK OK BAD OK OK BAD OK OK OK OK OK BAD BAD BAD...,OK OK OK OK OK BAD OK OK OK OK OK BAD OK OK OK...


In [72]:
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 
    'ours', 'ourselves', 'you', 'your', 'yours', 
    'yourself', 'yourselves', 'he', 'him', 'his', 
    'himself', 'she', 'her', 'hers', 'herself', 
    'it', 'its', 'itself', 'they', 'them', 'their', 
    'theirs', 'themselves', 'what', 'which', 'who', 
    'whom', 'this', 'that', 'these', 'those', 'am', 
    'is', 'are', 'was', 'were', 'be', 'been', 'being', 
    'have', 'has', 'had', 'having', 'do', 'does', 'did',
    'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or',
    'because', 'as', 'until', 'while', 'of', 'at', 
    'by', 'for', 'with', 'about', 'against', 'between',
    'into', 'through', 'during', 'before', 'after', 
    'above', 'below', 'to', 'from', 'up', 'down', 'in',
    'out', 'on', 'off', 'over', 'under', 'again', 
    'further', 'then', 'once', 'here', 'there', 'when', 
    'where', 'why', 'how', 'all', 'any', 'both', 'each', 
    'few', 'more', 'most', 'other', 'some', 'such', 'no', 
    'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 
    'very', 's', 't', 'can', 'will', 'just', 'don', 
    'should', 'now', '']
def synonym_replacement(words, n):
    new_words = words.copy()
    random_word_list = list(set([word for word in words if word not in stop_words]))
    random.shuffle(random_word_list)
    num_replaced = 0
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        if len(synonyms) >= 1:
            synonym = random.choice(list(synonyms))
            new_words = [synonym if word == random_word else word for word in new_words]
            print(new_words)
            print("replaced", random_word, "with", synonym)
            num_replaced += 1
        if num_replaced >= n: #only replace up to n words
            break

    #this is stupid but we need it, trust me
    sentence = ' '.join(new_words)
    new_words = sentence.split(' ')

    return new_words

def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word): 
        for l in syn.lemmas(): 
            synonym = l.name().replace("_", " ").replace("-", " ").lower()
            synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
            synonyms.add(synonym) 
    if word in synonyms:
        synonyms.remove(word)
    return list(synonyms)

In [31]:
text = 'disco was criticized as mindless , consumerist , overproduced and escapist. '
tokens = 'OK BAD OK OK BAD OK BAD OK OK OK BAD BAD'
text = text.split()
' '.join(synonym_replacement(text,4))

['disco', 'was', 'criticise', 'as', 'mindless', ',', 'consumerist', ',', 'overproduced', 'and', 'escapist.']
replaced criticized with criticise
['discotheque', 'was', 'criticise', 'as', 'mindless', ',', 'consumerist', ',', 'overproduced', 'and', 'escapist.']
replaced disco with discotheque
['discotheque', 'was', 'criticise', 'as', 'mindless', ',', 'consumerist', ',', 'overproduce', 'and', 'escapist.']
replaced overproduced with overproduce
['discotheque', 'was', 'criticise', 'as', 'senseless', ',', 'consumerist', ',', 'overproduce', 'and', 'escapist.']
replaced mindless with senseless


'discotheque was criticise as senseless , consumerist , overproduce and escapist.'

In [36]:
# lk = 'OK'
# lst = [lk] * 5
# lst.split()
thisdict = {
  "brand": "Ford",
  "year": 1964,
   "price":4545 
}
flag=0
for key, value in thisdict.items():
    
    if key=='year':
        flag+=1
    if flag ==1:
        thisdict[key] = value + 1 
print(thisdict)

{'brand': 'Ford', 'year': 1965, 'price': 4546}


In [21]:
augObj = DataAugmentation()
swapped_df = augObj.random_swap(df,2)

In [22]:
swapped_df

,source,target,src_tokens,tar_tokens
0,José Ortega y Gasset visited Husserl at Freibu...,1934 besuchte José Ortega y Gasset Husserl in ...,OK OK OK OK BAD OK OK OK BAD BAD OK,OK BAD OK BAD OK OK OK OK OK OK OK OK BAD OK O...
1,"however , a disappointing ninth in China meant...",eine enttäuschende Neunte in China bedeutete j...,OK BAD BAD BAD BAD OK OK OK OK OK OK OK OK OK ...,BAD BAD OK BAD OK BAD OK OK OK OK OK OK OK OK ...
2,"in his diary , Chase wrote that the release of...","in seinem Tagebuch , Chase schrieb , dass die ...",OK OK OK OK OK BAD OK OK BAD OK OK BAD OK BAD ...,OK OK OK OK OK OK OK BAD OK OK OK BAD OK OK OK...
3,Heavy arquebuses mounted on wagons were called...,schwere Arquebuses auf Waggons montiert wurden...,BAD BAD OK OK BAD BAD BAD OK OK OK OK,OK OK BAD BAD OK OK OK BAD OK OK BAD OK BAD OK...
4,once North Pacific salmon die off after spawni...,sobald der nordpazifische Lachs nach dem Laich...,OK OK OK OK OK OK OK BAD OK BAD BAD BAD BAD OK...,OK OK OK OK OK OK OK OK OK OK OK OK OK OK BAD ...
...,...,...,...,...
13995,the may also on or disallow unsanitary practic...,einige können auch unhygienische Praktiken wie...,BAD OK OK OK OK OK OK OK OK OK OK OK OK OK OK ...,OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK O...
13996,"in 1870s late 1860s , the the disappeared and ...",in den späten 1860er Jahren verschwanden die K...,OK OK OK OK OK OK BAD BAD OK OK BAD BAD OK OK ...,OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK B...
13997,"criticized was disco as mindless , consumerist...","Disco wurde als geistlos , konsumistisch , übe...",OK OK OK OK BAD BAD OK OK OK BAD OK OK,OK OK OK OK OK OK OK BAD OK OK OK BAD OK OK OK...
13998,planters would and fill large tobacco with hog...,die Pflanzer würden dann große Heuschrecken mi...,OK OK OK OK OK OK OK BAD BAD OK BAD BAD BAD BA...,OK OK OK OK OK BAD OK OK OK OK OK BAD OK OK OK...


In [23]:
augObj.random_deletion(df,0.3)

,source,target,src_tokens,tar_tokens
0,José Ortega y Gasset visited Husserl at Freibu...,1934 besuchte José Ortega y Gasset Husserl in ...,OK OK OK OK BAD OK OK OK BAD BAD OK,OK BAD OK BAD OK OK OK OK OK OK OK OK BAD OK O...
1,"however , a disappointing ninth in China meant...",eine enttäuschende Neunte in China bedeutete j...,OK BAD BAD BAD BAD OK OK OK OK OK OK OK OK OK ...,BAD BAD OK BAD OK BAD OK OK OK OK OK OK OK OK ...
2,"in his diary , Chase wrote that the release of...","in seinem Tagebuch , Chase schrieb , dass die ...",OK OK OK OK OK BAD OK OK BAD OK OK BAD OK BAD ...,OK OK OK OK OK OK OK BAD OK OK OK BAD OK OK OK...
3,Heavy arquebuses mounted on wagons were called...,schwere Arquebuses auf Waggons montiert wurden...,BAD BAD OK OK BAD BAD BAD OK OK OK OK,OK OK BAD BAD OK OK OK BAD OK OK BAD OK BAD OK...
4,once North Pacific salmon die off after spawni...,sobald der nordpazifische Lachs nach dem Laich...,OK OK OK OK OK OK OK BAD OK BAD BAD BAD BAD OK...,OK OK OK OK OK OK OK OK OK OK OK OK OK OK BAD ...
...,...,...,...,...
13995,some may also or disallow unsanitary such as k...,einige können auch unhygienische Praktiken wie...,OK OK OK OK OK OK OK OK OK OK BAD OK,OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK O...
13996,"the late , the crinolines disappeared and the ...",in den späten 1860er Jahren verschwanden die K...,OK OK OK OK OK BAD OK BAD BAD OK OK OK OK,OK OK OK OK OK OK OK OK OK OK OK OK OK OK OK B...
13997,"disco criticized as , consumerist , and escapi...","Disco wurde als geistlos , konsumistisch , übe...",OK OK OK BAD OK OK BAD OK OK,OK OK OK OK OK OK OK BAD OK OK OK BAD OK OK OK...
13998,planters would then hogsheads with tobacco con...,die Pflanzer würden dann große Heuschrecken mi...,OK OK BAD BAD OK OK OK BAD BAD BAD OK,OK OK OK OK OK BAD OK OK OK OK OK BAD OK OK OK...
